In [1]:
import os
from os.path import join, exists
import sys
import pandas as pd
import numpy as np
import childespy

R[write to console]: Installing package into ‘/home/stephan/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)



Reinstalling childesr version 0.2.1


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/childesr_0.2.3.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 22865 bytes (22 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to conso

In [2]:
sys.path.append('../../')
from src.utils import load_splits, load_models
from src.analysis import examples_figure
from src.utils import sample_models_across_time, configuration
config = configuration.Config()


%load_ext autotime
%load_ext line_profiler
%pdb

Automatic pdb calling has been turned ON
time: 3.18 ms (started: 2023-03-05 13:59:18 -08:00)


# Success Example

In [3]:
# Make this config regenerate controlled later

pvd_idx = childespy.get_sql_query('select * from corpus where name = "Providence"', db_version = "2020.1").iloc[0]['id']

regenerate = False
this_path = join(config.prov_csv_dir, 'pvd_utt_glosses.csv')

if regenerate:
    utt_glosses = childespy.get_sql_query('select gloss, transcript_id, id, \
    utterance_order, speaker_code, type from utterance where corpus_id = '+str(pvd_idx) ,
        db_version = "2020.1")
    utt_glosses.to_csv(this_path, index=False)
else: 
    utt_glosses = pd.read_csv(this_path)

R[write to console]: Using supported database version: '2020.1'.



time: 2.12 s (started: 2023-03-05 13:59:18 -08:00)


## Load phonology and context

In [4]:
all_tokens_phono = load_splits.load_phono()

time: 7.23 s (started: 2023-03-05 13:59:20 -08:00)


In [5]:
success_idx = 16928243

all_tokens_phono.loc[all_tokens_phono.utterance_id == success_idx][['gloss','actual_phonology_no_dia',
 'model_phonology_no_dia', 'utterance_id','bert_token_id','utterance_order','transcript_id']]


,gloss,actual_phonology_no_dia,model_phonology_no_dia,utterance_id,bert_token_id,utterance_order,transcript_id
997717,I want to read,,,16928243,997717,310,42336
997718,I want to read,aə,aə,16928243,997718,310,42336
997719,I want to read,wɑn,wɑnt,16928243,997719,310,42336
997720,I want to read,də,tu,16928243,997720,310,42336
997721,I want to read,wid,ɹid,16928243,997721,310,42336
997722,I want to read,,,16928243,997722,310,42336


time: 27.3 ms (started: 2023-03-05 13:59:28 -08:00)


In [6]:
target_transcript_id = 42336 # Corresponds to the success_idx 

time: 311 µs (started: 2023-03-05 13:59:28 -08:00)


In [7]:
# Broader context of sentences
utt_glosses.loc[(utt_glosses.transcript_id == target_transcript_id) &
                (utt_glosses.utterance_order.isin(range(310-10, 310+2)))] # Note to self: changed the range here

,gloss,transcript_id,id,utterance_order,target_child_name,speaker_code,type
162628,Jasmine,42336,16928069,300,Lily,MOT,declarative
162639,a ballet,42336,16928081,301,Lily,CHI,declarative
162656,is Jasmine a ballerina,42336,16928098,302,Lily,MOT,question
162670,yeah,42336,16928113,303,Lily,CHI,declarative
162686,oh I didn't know that,42336,16928129,304,Lily,MOT,declarative
162710,I ready now yyy,42336,16928154,305,Lily,CHI,declarative
162730,whoa,42336,16928174,306,Lily,MOT,declarative
162744,yyy,42336,16928189,307,Lily,CHI,declarative
162760,yyy yyy,42336,16928205,308,Lily,CHI,declarative
162779,you want mamma let's see,42336,16928225,309,Lily,MOT,declarative


time: 50.2 ms (started: 2023-03-05 13:59:28 -08:00)


## Estimate Prior and Posterior Probability

In [13]:
import imp
imp.reload(examples_figure)
imp.reload(sample_models_across_time)
imp.reload(load_models)

/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/analysis/examples_figure.py:3: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/stephan/notebooks/child-directed-listening/config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/sample_models_across_time.py:11: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/stephan/notebooks/child-directed-listening/config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/load_models.py:12: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/stephan/notebooks/child-directed-listening/config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()


<module 'src.utils.load_models' from '/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/load_models.py'>

time: 9.71 ms (started: 2023-03-05 16:40:03 -08:00)


In [14]:
# how to select the models to run under the new system

time: 501 µs (started: 2023-03-05 16:40:04 -08:00)


In [15]:
model_args = load_models.gen_all_model_args()

time: 937 µs (started: 2023-03-05 16:40:05 -08:00)


In [16]:
model_args

[{'training_split': 'adult-written',
  'training_dataset': 'all',
  'model_type': 'BERT',
  'use_tags': False,
  'context_width': 0},
 {'training_split': 'adult-written',
  'training_dataset': 'all',
  'model_type': 'BERT',
  'use_tags': False,
  'context_width': 20},
 {'training_split': 'adult-written',
  'training_dataset': 'all',
  'model_type': 'BERT',
  'use_tags': False,
  'context_width': [20, 0]},
 {'training_split': 'Providence',
  'training_dataset': 'all',
  'model_type': 'GPT-2',
  'contextualized': 1,
  'use_tags': True,
  'context_width': 0},
 {'training_split': 'Providence',
  'training_dataset': 'all',
  'model_type': 'GPT-2',
  'contextualized': 1,
  'use_tags': True,
  'context_width': 20},
 {'training_split': 'Providence',
  'training_dataset': 'all',
  'model_type': 'GPT-2',
  'contextualized': 1,
  'use_tags': True,
  'context_width': [20, 0]},
 {'training_split': 'Providence',
  'training_dataset': 'all',
  'model_type': 'GPT-2',
  'contextualized': 1,
  'use_tags

time: 13.8 ms (started: 2023-03-05 16:40:05 -08:00)


In [17]:
raw_scores_across_models = examples_figure.get_scores_across_models(success_idx, model_args, True)

/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/paths.py:75: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/stephan/notebooks/child-directed-listening/config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/paths.py:75: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/stephan/notebooks/child-directed-listening/config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM

Running model eval_adult-written_all_no_tags_BERT_Providence_all_0... at age 0
Computing failure scores
Computing success scores
Getting the Levenshtein distance matrix
Getting posteriors
Running model eval_adult-written_all_no_tags_BERT_Providence_all_0... at age 0
Computing failure scores
Computing success scores


/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tail[1] = -1 * np.log(1)
/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tail[2] = [int(x) if not np.isnan(x) else '' for x in tail[2]]
/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:313: SettingWithCopyWarning: 
A value is trying t

Loaded transducer
Getting posteriors


/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/paths.py:75: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/stephan/notebooks/child-directed-listening/config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/paths.py:75: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/stephan/notebooks/child-directed-listening/config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM

...

ipdb>  quit()


BdbQuit: 

> /usr/lib/python3.8/bdb.py(113)dispatch_line()
    111         if self.stop_here(frame) or self.break_here(frame):
    112             self.user_line(frame)
--> 113             if self.quitting: raise BdbQuit
    114         return self.trace_dispatch
    115 



ipdb>  quit()


time: 1min 29s (started: 2023-03-05 16:40:06 -08:00)


In [ ]:
# [X] figure out why the token is double masked in this case and therefore not found.
# how is this different than when this is called from the command line in the regular fashion
# utt_df.loc[utt_df.token == '[MASK]'] #the more robust way to do this is with token_id = 103
# [ ] An issue with the FST
#    df[[4]] = -1 * np.log(normalize_log_probs(df[[4]]))

In [ ]:
scores_across_models = pd.concat(raw_scores_across_models)

In [ ]:
np.unique(scores_across_models.model)

In [ ]:
scores_across_models = scores_across_models.loc[scores_across_models.likelihood_type == 'wfst']

In [ ]:
childes_all_title = 'eval_Providence_all_with_tags_BERT_Providence_all_20'
adult_all_title = 'eval_adult-written_all_no_tags_BERT_Providence_all_20'
unigram_title = 'eval_Providence_all_no_tags_data_unigram_Providence_all_0'
flat_title = 'eval_no-split_no-dataset_no_tags_flat_unigram_Providence_all_0'

In [ ]:
success_example = scores_across_models.loc[(scores_across_models.model == childes_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','posterior_probability','token','likelihood_type','prior_rank','posterior_rank']]
success_example

In [ ]:
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
print('CDL+Context Prior')
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
print('CDL+Context Posterior')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],3))+')' for i in range(len(words))]) 

In [ ]:
success_example = scores_across_models.loc[(scores_across_models.model == adult_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
success_example

print('BERT+Context Prior')
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
adult_all_title

In [ ]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
print('BERT+Context Posterior')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],4))+')' for i in range(len(words))]) 

In [ ]:
success_example = scores_across_models.loc[(scores_across_models.model == unigram_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token', 'prior_rank', 'posterior_rank']]
#print(success_example)
print('CHILDES-1Gram Prior')
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
print('CHILDES-1gram Posterior')
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
success_example = scores_across_models.loc[(scores_across_models.model == flat_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
success_example
print('UniformPrior Prior')
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
print('UniformPrior Posterior')
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

# Failure Example

In [ ]:
yyy_idx = 16813515

raw_scores_across_models = examples_figure.get_scores_across_models(yyy_idx, model_args, False)
scores_across_models = pd.concat(raw_scores_across_models)

In [ ]:
all_tokens_phono.loc[all_tokens_phono.utterance_id == yyy_idx][['gloss','actual_phonology_no_dia',
 'model_phonology_no_dia', 'utterance_id','bert_token_id','utterance_order','transcript_id']]

In [ ]:
scores_across_models

In [ ]:
yyy_example = scores_across_models.loc[(scores_across_models.model == childes_all_title) & (scores_across_models.likelihood_type == 'wfst')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token','likelihood_type','posterior_rank']]
yyy_example

In [ ]:
words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))])

In [ ]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
yyy_example = scores_across_models.loc[(scores_across_models.model == adult_all_title ) & (scores_across_models.likelihood_type == 'wfst')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
yyy_example = scores_across_models.loc[(scores_across_models.model == unigram_title) & (scores_across_models.likelihood_type == 'wfst')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
yyy_example = scores_across_models.loc[(scores_across_models.model == flat_title) & (scores_across_models.likelihood_type == 'wfst')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
# Get the surrounding context
utt_glosses.loc[(utt_glosses.transcript_id == 42253) &
                (utt_glosses.utterance_order.isin(range(112-3, 112+3)))]